In [ ]:
%matplotlib inline
import os
import re
from openquake.man.checking_utils.source_model_utils import read
from openquake.mbt.oqt_project import OQtProject, OQtSource

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
faults_xml_filename = os.path.join(oqtkp.directory, getattr(model,'faults_xml_filename'))
print(os.path.realpath(faults_xml_filename))

## Get shallow crust fault sources

In [ ]:
sources, _ = read(faults_xml_filename)
for f in sources:
    sid = str(f.source_id)
    if not re.search('^fs_', sid):
        sid = 'fs_{:s}'.format(sid)
    src = OQtSource(sid, 'SimpleFaultSource')
    src.trace = f.fault_trace
    src.msr = f.magnitude_scaling_relationship
    src.mfd = f.mfd
    src.rupture_aspect_ratio = f.rupture_aspect_ratio
    src.trt = f.tectonic_region_type
    src.dip = f.dip
    src.upper_seismogenic_depth = f.upper_seismogenic_depth
    src.lower_seismogenic_depth = f.lower_seismogenic_depth
    src.name = f.name
    src.rake = f.rake
    model.sources[sid] = src

## Update project

In [ ]:
oqtkp.models[model_id] = model
oqtkp.save()